In [80]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
le=WordNetLemmatizer()
tf=TfidfVectorizer()
stemmer=PorterStemmer()

In [81]:
df=pd.read_csv('Data.csv',encoding = "ISO-8859-1")
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101 entries, 0 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4101 non-null   object
 1   Label   4101 non-null   int64 
 2   Top1    4101 non-null   object
 3   Top2    4101 non-null   object
 4   Top3    4101 non-null   object
 5   Top4    4101 non-null   object
 6   Top5    4101 non-null   object
 7   Top6    4101 non-null   object
 8   Top7    4101 non-null   object
 9   Top8    4101 non-null   object
 10  Top9    4101 non-null   object
 11  Top10   4101 non-null   object
 12  Top11   4101 non-null   object
 13  Top12   4101 non-null   object
 14  Top13   4101 non-null   object
 15  Top14   4101 non-null   object
 16  Top15   4101 non-null   object
 17  Top16   4101 non-null   object
 18  Top17   4101 non-null   object
 19  Top18   4101 non-null   object
 20  Top19   4101 non-null   object
 21  Top20   4101 non-null   object
 22  Top21   4101 non-null   

In [83]:
### Spliting Data into Train and Test 
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [84]:
text_data=train.iloc[:,2:27]

In [85]:
### Rename columns 
l=[i for i in range(0,25)]
new_columns=[str(i) for i in l]
text_data.columns=new_columns

In [86]:
### Remove punctuations
text_data.replace('[^a-zA-Z]'," ",regex=True,inplace=True)

In [87]:
### # Convert data to lower case
for i in new_columns:
    text_data[i]=text_data[i].str.lower()

In [88]:
text_data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [89]:
### Joining all columns data into single sentence
headlines = []
for row in range(0,len(text_data.index)):
    headlines.append(' '.join(str(x) for x in text_data.iloc[row,0:25]))

In [90]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [91]:
traindataset=tf.fit_transform(headlines)

In [92]:
traindataset

<3975x46002 sparse matrix of type '<class 'numpy.float64'>'
	with 817766 stored elements in Compressed Sparse Row format>

In [93]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=200,criterion='entropy')

In [94]:
rf.fit(traindataset,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [95]:
test.index

Int64Index([3723, 3724, 3725, 3726, 3727, 3728, 3729, 3730, 3731, 3732,
            ...
            4091, 4092, 4093, 4094, 4095, 4096, 4097, 4098, 4099, 4100],
           dtype='int64', length=378)

In [96]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = tf.transform(test_transform)

In [97]:
test_dataset

<378x46002 sparse matrix of type '<class 'numpy.float64'>'
	with 115691 stored elements in Compressed Sparse Row format>

In [99]:
pred=rf.predict(test_dataset)

In [101]:
report=classification_report(test['Label'],pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.74      0.82       186
           1       0.79      0.93      0.85       192

    accuracy                           0.84       378
   macro avg       0.85      0.83      0.83       378
weighted avg       0.85      0.84      0.83       378



In [102]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [110]:
lr.fit(traindataset,train['Label'])

LogisticRegression()

In [111]:
pred_lr=lr.predict(test_dataset)

In [112]:
report=classification_report(test['Label'],pred_lr)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.67      0.77       186
           1       0.74      0.93      0.83       192

    accuracy                           0.80       378
   macro avg       0.82      0.80      0.80       378
weighted avg       0.82      0.80      0.80       378



In [117]:
from sklearn.svm import SVC
SVM=SVC()

In [118]:
SVM.fit(traindataset,train['Label'])

SVC()

In [119]:
pred_SVM=lr.predict(test_dataset)

In [120]:
report=classification_report(test['Label'],pred_SVM)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.67      0.77       186
           1       0.74      0.93      0.83       192

    accuracy                           0.80       378
   macro avg       0.82      0.80      0.80       378
weighted avg       0.82      0.80      0.80       378

